In [19]:

import os 
import numpy as np
import pandas as pd
import fiona
import dask.array as da
import rasterio
import rasterio.mask
import xarray as xr
import rioxarray
import geopandas as gpd
import shapely.geometry as geoms
from shapely.geometry import box

from rasterio.mask import mask
from rasterio.plot import show
from rasterio.merge import merge  
from rasterio.warp import calculate_default_transform, reproject, Resampling
from os import path 
from pathlib import Path
from glob import glob
from osgeo import gdal, osr

import logging, sys
import concurrent.futures
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient
from azure.storage.blob import ContainerClient
import tqdm
import rasterstats



In [21]:

year              = 2021
tiles             = ['30SWG', '30SWH','30SXG', '30SXH', '30SXJ', '30SYH'] 
months            = ['01' ,'02','03','04','05','06','07','08','09','10','11', '12']
days              = ['01' ,'02','03','04','05','06','07', '08', '09','10','11','12', '13','14','15','16','17','18','19','20','21','22','23','24','25','26', '27','28','29','30','31' ]

#
CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=ccubed;AccountKey=b284EE1qXogSUEhsCmb3Z9m2YrcCGW3m+PsRdMlhguCOVLRoJKA+s6vQL61kMv/5mpUH5MFUsJczH8Dgo5jxhA==;EndpointSuffix=core.windows.net"
name              = "_REFL.tif"

directory         = f"../Data/Crop_Classification/S2/{year}/"
indices           = ['NDVI', 'SAVI', 'NDMI'] 

parcels           = '../Data/Crop_Classification/Crop_parcel_2021/Cultivos_regados_AH2021_v6.shp'

Q3                = ['07','08', '09']
indices_Q3        = ['NDVI', 'SAVI']
percetiles        = [10, 20, 50 , 90 , 95 ] 


In [22]:

# Number of files
for tile in tiles:
        trg_path = Path(directory).joinpath(tile)
        files = glob(f'{trg_path}/*/*.tif')
        print(tile, len(files))
        

30SWG 663
30SWH 674
30SXG 594
30SXH 614
30SXJ 619
30SYH 624


# Create a folder structure

In [77]:
for tile in tiles:
    for month in months:
        trg_path = Path(directory).joinpath(tile)
        if not trg_path.joinpath(month).is_dir(): 
            trg_path.joinpath(month).mkdir(parents=True) 
    print(trg_path)

../Data/Crop_Classification/S2/2021/30SWG
../Data/Crop_Classification/S2/2021/30SWH
../Data/Crop_Classification/S2/2021/30SXG
../Data/Crop_Classification/S2/2021/30SXH
../Data/Crop_Classification/S2/2021/30SXJ
../Data/Crop_Classification/S2/2021/30SYH


# Bulk download of S2 Scenes from Azure Storage (ca.1h - ca. 1year)

In [78]:
def down_name(tile):
        for month in months:
            for day in days:
                container = ContainerClient.from_connection_string(conn_str=CONNECTION_STRING, container_name="ccc-storage")
                prefix = f"Sentinel-2/L2B/T{tile}/{year}/{month}/{day}/"
                blob_list = list(container.list_blobs(name_starts_with=prefix))
                for blob in tqdm.tqdm(blob_list, disable=None, leave=False):
                    if name in blob.name:
                        file = blob.name.split('/')[6] 
                        download_file_path = os.path.join(directory,tile,month,file)
                        if not path.isfile(download_file_path): 
                            print("Downloading:", file)
                            with open(download_file_path, "wb") as download_file:
                                download_file.write(container.download_blob(blob,max_concurrency = 50).readall())
                        #else:
                            #print("Exist:",file)
def run(tiles):
    with ThreadPool(processes=int(2)) as pool:
        return pool.map(down_name, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")

Downloading: S2B_MSIL2A_20210619T104619_N9999_R051_T30SWG_20220707T081456_REFL.tif
Done


# Calculation NDVI, SAVI, NDMI per each scenes

- NDVI (Sentinel 2) = (B8 – B4) / (B8 + B4)
- SAVI (Sentinel 2) = (B08 – B04) / (B08 + B04 + 0.428) * (1.428)
- NDMI (Sentinel 2) = (B8 – B11) / (B8 + B11)


File_REFL {B2,B3,B4,B5,B6,B7,B8A,B11,B12


//https://github.com/DHI-GRAS/sen-et-input-scripts/blob/master/senet_input_creator/sentinel2_inputs.py

In [85]:
def calc_indices(tile):
    for month in months:
        files = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*REFL.tif')
        for file in files:   
           
            output_ndvi = ('{}_NDVI.tif'.format(file.split('\\')[-1][:-9]))
            output_savi = ('{}_SAVI.tif'.format(file.split('\\')[-1][:-9]))
            output_ndmi = ('{}_NDMI.tif'.format(file.split('\\')[-1][:-9]))
            
            if not path.isfile(output_ndmi):
                with rasterio.open(file) as src:
                    
                     band_red = src.read(3)
                     band_nir = src.read(7)
                     band_swir = src.read(8)
                        
                     ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))
                     savi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float) + 0.428) * (1.428)   
                     ndmi = (band_nir.astype(float) - band_swir.astype(float)) / (band_nir.astype(float) + band_swir.astype(float)) 
                  
                     # Set spatial characteristics of the output object to mirror the input
                     kwargs = src.meta
                     kwargs.update(
                             dtype=rasterio.float32,
                             count = 1)
                        
                     # Create the files
                     with rasterio.open(output_ndvi, 'w', **kwargs) as dst:
                         dst.write_band(1, ndvi.astype(rasterio.float32))
                        
                     with rasterio.open(output_savi, 'w', **kwargs) as dst:
                         dst.write_band(1, savi.astype(rasterio.float32))
                                
                     with rasterio.open(output_ndmi, 'w', **kwargs) as dst:
                         dst.write_band(1, ndmi.astype(rasterio.float32))

def run(tiles):
    with ThreadPool(processes=int(10)) as pool:
        return pool.map(calc_indices, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
        

Done


# Create an annual raster stack of NDVI, SAVI, NDMI for each tile

In [9]:

def stack_indices(tile):
    for index in indices:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/*/*{index}.tif')
        print(tile, index, len(file_list))
      
        with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
        
        meta.update(count = len(file_list))
        
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}.tif'

        if not path.isfile(output):
            with rasterio.open(output, 'w', **meta) as dst:
                for id, layer in enumerate(file_list, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
                        meta = src1.meta
                        print(id, src1)
                print(output)
                
def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_indices, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
        
        

30SWG NDVI 128
30SWG SAVI 128
30SWG NDMI 128
30SWH NDVI 130
30SWH SAVI 130
30SWH NDMI 130
30SXG NDVI 114
30SXG SAVI 114
30SXG NDMI 114
30SXH NDVI 118
30SXH SAVI 118
30SXH NDMI 118
30SXJ NDVI 119
30SXJ SAVI 119
30SXJ NDMI 119
30SYH NDVI 120
30SYH SAVI 120
30SYH NDMI 120
Done


# Clip the raster stack 

In [10]:

with fiona.open('../Data/AOI/Rio_Segura_AOI_Tighter25830.geojson', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
def clip_raster(tile):
    for index in indices:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}.tif')
        print(tile, index, len(file_list))
        for id, layer in enumerate(file_list, start=1):
            output = f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_clip.tif'
            if not path.isfile(output):
                print(output)
                with rasterio.open(layer) as src:
                    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    out_meta = src.meta

                # Save clipped imagery and change dtype
                out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform,
                         'dtype': 'float32',
                         'compress':'lzw'})

                with rasterio.open(output, "w", **out_meta) as dest:
                    dest.nodata = 0
                    dest.write(out_image)
                    print(output)

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(clip_raster, tiles)
        
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")

30SWG NDVI 1
30SWG SAVI 1
30SWG NDMI 1
30SWH NDVI 1
30SWH SAVI 1
30SWH NDMI 1
30SXG NDVI 1
30SXG SAVI 1
30SXG NDMI 1
30SXH NDVI 1
30SXH SAVI 1
30SXH NDMI 1
30SXJ NDVI 1
30SXJ SAVI 1
30SXJ NDMI 1
30SYH NDVI 1
30SYH SAVI 1
30SYH NDMI 1
Done


# Calculation of Percentile

In [11]:

def percentile(tile):
    for index in indices:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}.tif')
        print(tile, index, len(file_list))
        for percentile in percetiles:
            output = f'../Data/Crop_Classification/S2/{year}/{tile}/{index}_p{percentile}.tif' 
            if not path.isfile(output):
                print(percentile)
                ds  = xr.open_rasterio(file_list[0])
                percentil = ds.quantile(percentile/100, dim='band') 
                print(percentile/100)
                print(output)
                percentil.rio.to_raster(output)
                input_raster = gdal.Open(output)
                gdal.Warp(output, input_raster, dstSRS="EPSG:25830")
    

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(percentile, tiles)
         
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
    

30SWG NDVI 1
30SWG SAVI 1
30SWG NDMI 1
30SWH NDVI 1
30SWH SAVI 1
30SWH NDMI 1
30SXG NDVI 1
30SXG SAVI 1
30SXG NDMI 1
30SXH NDVI 1
30SXH SAVI 1
30SXH NDMI 1
30SXJ NDVI 1
30SXJ SAVI 1
30SXJ NDMI 1
30SYH NDVI 1
30SYH SAVI 1
30SYH NDMI 1
Done


# Check all percentile raster

In [12]:

for tile in tiles:
    list = []
    for index in indices:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{index}_p*_UTM30.tif')
        list.extend(file_list)      
    print(tile, len(list))
    

30SWG 0
30SWH 0
30SXG 0
30SXH 0
30SXJ 0
30SYH 0


# Raster stack of Q3 NDVI and SAVI (Jul, Aug, Sep)


In [13]:

def stack_indices_q3(tile):
    for index in indices_Q3:
        list = []
        for month in Q3:
            file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*{index}.tif')
            list.extend(file_list)    
        print(tile, index, len(list))
        with rasterio.open(list[0]) as src0:
            meta = src0.meta
        meta.update(count = len(list))
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_q3.tif'
        if not path.isfile(output):
            with rasterio.open(output, 'w', **meta) as dst:
                 for id, layer in enumerate(list, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
                    print(output)
                
def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_indices_q3, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
        

30SWG NDVI 37
30SWG SAVI 37
30SWH NDVI 37
30SWH SAVI 37
30SXG NDVI 35
30SXG SAVI 35
30SXH NDVI 35
30SXH SAVI 35
30SXJ NDVI 36
30SXJ SAVI 36
30SYH NDVI 34
30SYH SAVI 34
Done


# Median raster from raster stack of  Q3 NDVI and SAVI (Jul, Aug, Sep)


In [14]:
def stack_indices_q3_median(tile):
    for index in indices_Q3:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/*{index}_q3.tif')
        print(tile, index, len(file_list))
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/{index}_q3_median.tif'
        if not path.isfile(output):
            ds  = xr.open_rasterio(file_list[0])
            mean = ds.mean(dim='band')
            mean.rio.to_raster(output)   
            print(output)
            ds = gdal.Open(output, 1)
            sr = osr.SpatialReference()
            sr.ImportFromEPSG(25830)
            ds.SetProjection(sr.ExportToWkt())
            ds = None

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_indices_q3_median, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")


30SWG NDVI 1
30SWG SAVI 1
30SWH NDVI 1
30SWH SAVI 1
30SXG NDVI 1
30SXG SAVI 1
30SXH NDVI 1
30SXH SAVI 1
30SXJ NDVI 1
30SXJ SAVI 1
30SYH NDVI 1
30SYH SAVI 1
Done


# Calculation of amp5095 Amplitude for SAVI, NDMI and NDVI, 
 - savi_amp5095 = savi annual percentile 95 subtract percentile 50
 - ndvi_amp5095  = ndvi  annual percentile 95 subtract percentile 50
 - ndmi_amp5095  = ndvi  annual percentile 95 subtract percentile 50

In [15]:
def stack_amp5095(tile):
    for index  in indices:
        print(tile, index)
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/{index}_amp5095.tif'   
        if not path.isfile(output):
            p95  = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_p95_UTM30.tif')
            p50  = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_p50_UTM30.tif')
            amp_5095 = xr.open_rasterio(p95[0]) - xr.open_rasterio(p50[0]) 
            amp_5095.rio.to_raster(output)
            input_raster = gdal.Open(output)
            gdal.Warp(output, input_raster, dstSRS="EPSG:25830")
            print(output)

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_amp5095, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
   

30SWG NDVI
30SWG SAVI
30SWG NDMI
30SWH NDVI
30SWH SAVI
30SWH NDMI
30SXG NDVI
30SXG SAVI
30SXG NDMI
30SXH NDVI
30SXH SAVI
30SXH NDMI
30SXJ NDVI
30SXJ SAVI
30SXJ NDMI
30SYH NDVI
30SYH SAVI
30SYH NDMI
Done


# Copernicus GLO-30 Digital Elevation Model resample to S2 

In [16]:

def openRaster(raster):
    closeOnExit = False
    try:
        raster.GetProjection()
        openRaster = raster
    except AttributeError:
        openRaster = gdal.Open(raster)
        closeOnExit = True
    return openRaster, closeOnExit



def getRasterInfo(raster):
    r, closeOnExit = openRaster(raster)
    proj = r.GetProjection()
    gt = r.GetGeoTransform()
    sizeX = r.RasterXSize
    sizeY = r.RasterYSize
    extent = [gt[0], gt[3]+gt[5]*sizeY, gt[0]+gt[1]*sizeX, gt[3]]
    bands = r.RasterCount
    if closeOnExit:
        r = None
  
    return proj, gt, sizeX, sizeY, extent, 

def resampleWithGdalWarp(srcFile, templateFile, outFile="", outFormat="MEM",
                         resampleAlg="average"):
    # Get template projection, extent and resolution
    proj, gt, sizeX, sizeY, extent= getRasterInfo(templateFile)

    # Resample with GDAL warp
    outDs = gdal.Warp(outFile,
                      srcFile,
                      format=outFormat,
                      dstSRS='EPSG:25830',
                      xRes=gt[1],
                      yRes=gt[5],
                      outputBounds=extent,
                      resampleAlg=resampleAlg)

import gdal 

for tile in tiles:
    templateFile = f'../Data/Crop_Classification/S2/{year}/{tile}/stack_NDMI_p90.tif'
    glist = glob(f'../Data/Crop_Classification/elevation.tif')
                 
    for raster in glist:
        output_file = os.path.join(os.path.dirname(raster),'S2', str(year), tile,'{}_20m.tif'.format(os.path.basename(raster).split('\\')[-1][:-4])) 
        if not path.isfile(output_file):
            print(output_file)
            resampleWithGdalWarp(raster, templateFile, outFile=output_file, outFormat="GTiff", resampleAlg="average")
        
        

In [17]:
for month in months:
    src_files_to_mosaic = []
    month_tiles_list = glob(f'../Data/Crop_Classification/S2/{year}/*/elevation_20m.tif')
    print(month_tiles_list)
    for file_month in month_tiles_list:
        src = rasterio.open(file_month)
        src_files_to_mosaic.append(src) 
        
    mosaic, out_trans = merge(src_files_to_mosaic)

    profile = src.meta.copy()
    profile.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans,
                    "compress": "lzw"})
    

    out_file  = f'../Data/Crop_Classification/S2/{year}/elevation_GLO_25830.tif'
    
    if not path.isfile(out_file):
        with rasterio.open(out_file, "w", **profile) as dest:
            print(out_file)
            dest.write(mosaic)
            
    ds = gdal.Open(out_file, 1)
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(25830)
    ds.SetProjection(sr.ExportToWkt())
    ds = None


['../Data/Crop_Classification/S2/2021/30SWG/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SWH/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXG/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXH/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXJ/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SYH/elevation_20m.tif']
../Data/Crop_Classification/S2/2021/elevation_GLO_25830.tif
['../Data/Crop_Classification/S2/2021/30SWG/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SWH/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXG/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXH/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXJ/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SYH/elevation_20m.tif']
['../Data/Crop_Classification/S2/2021/30SWG/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SWH/elevation_20m.tif', '../Data/Crop_Classification/S2/2021/30SXG/elevation_20

# Calculate indices: HP_prob, CO_prob, AyF_prob
 - HP_prob (savi_amp5095, ndvi_p10)
      -  HP_prop = expression('1 / (1 + exp(- (-2.37 + (-8.19 * savi_amp5095) + (7.084 * ndvi_p10)))))
          - savi_amp5095 = savi annual percentile 95 subtract percentile 50
     

 - CO_prob (ndmi_p90, elevation), 
     -  CO_prob = expression('1 / (1 + exp(- (0.11 + (-7.5 * ndmi_p90) + (0.006 * elevation)))))
     

 - AyF_prob(ndmi_p95, ndvi_p95, elevation)
     - AyF_prob = expression('1 / (1 + exp(- (0.05 + (1.5 * ndmi_p95) + (3.08 * ndvi_p95) + (-0.003 * elevation))))) 



In [33]:

indices_prob = ['HP', 'CO', 'AyF']

def Prob_index(tile):
    
     for index  in indices_prob:
        print(tile, index)
        
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/{index}_prob.tif'
        
        if not path.isfile(output):
            
            elev          = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/elevation_20m.tif')
            savi_amp5095  = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/SAVI_amp5095.tif')
            ndvi_p10      = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/NDVI_p10.tif')
            ndvi_p95      = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/NDVI_p95.tif')
            ndmi_p90      = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/NDMI_p90.tif')
            ndmi_p95      = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/NDMI_p95.tif')

            print(savi_amp5095, ndvi_p10, ndvi_p95, ndmi_p90, ndmi_p95, elev)

            with rasterio.open(elev[0]) as src: elev = src.read(1)                             
            with rasterio.open(savi_amp5095[0],'r+') as src: savi_amp5095 = src.read(1); src.nodata = 0   
            with rasterio.open(ndvi_p10[0],'r+') as src: ndvi_p10 = src.read(1); src.nodata = 0
            with rasterio.open(ndvi_p95[0],'r+') as src: ndvi_p95 = src.read(1); src.nodata = 0 
            with rasterio.open(ndmi_p90[0],'r+') as src: ndmi_p90 = src.read(1); src.nodata = 0
            with rasterio.open(ndmi_p95[0],'r+') as src: ndmi_p95 = src.read(1); src.nodata = 0

            kwargs = src.meta 
            kwargs.update({"driver": "GTiff", 'compress':'lzw'})

            if 'HP' == index:
                data  = (1 / (1 + np.exp(- (-2.37 + (-8.19 * savi_amp5095.astype(float)) + (7.084 * ndvi_p10.astype(float))))))
            if 'CO' == index:
                data  = (1 / (1 + np.exp(- (0.11 + (-7.5 * ndmi_p90.astype(float)) + (0.006 * elev.astype(float))))))
            if 'AyF' == index:
                data  = (1 / (1 + np.exp(- (0.05 + (1.5 * ndmi_p95.astype(float)) + (3.08 * ndvi_p95.astype(float) + (-0.003 * elev.astype(float)))))))
            
            print(index, output)

            with rasterio.open(output, 'w', **kwargs) as dst:
                print('writing')
                dst.nodata = 0
                dst.write_band(1, data)
            print('Reprojecting')    
            ds = gdal.Open(output, 1)
            sr = osr.SpatialReference()
            sr.ImportFromEPSG(25830)
            ds.SetProjection(sr.ExportToWkt())

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(Prob_index, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
   

30SWG HP
30SWG CO
30SWG AyF
30SWH HP
30SWH CO
30SWH AyF
30SXG HP
30SXG CO
30SXG AyF
30SXH HP
30SXH CO
30SXH AyF
30SXJ HP
30SXJ CO
30SXJ AyF
30SYH HP
30SYH CO
30SYH AyF
Done


# Forest Density Layer

In [26]:
src_files_to_mosaic = []
month_tiles_list = glob(f'../Data/Crop_Classification/forest_density_TCD/*.tif')
print(month_tiles_list)
for file_month in month_tiles_list:
    src = rasterio.open(file_month)
    src_files_to_mosaic.append(src) 
    
mosaic, out_trans = merge(src_files_to_mosaic)

profile = src.meta.copy()
profile.update({"driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "compress": "lzw"})

out_file  = f'../Data/Crop_Classification/forest_density_TCD.tif'

if not path.isfile(out_file):
    with rasterio.open(out_file, "w", **profile) as dest:
        dest.write(mosaic)
        
input_ras = gdal.Open(out_file)
output_ras = r'../Data/Crop_Classification/forest_density_TCD_25830.tif'
options = gdal.WarpOptions(format = 'GTiff',  creationOptions = ['TFW=YES', 'COMPRESS=LZW'])

warp = gdal.Warp(output_ras, input_ras, dstSRS='EPSG:25830', options = options)
warp = None # Closes the files


['../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E31N17_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E31N18_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E32N16_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E32N17_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E32N18_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E33N16_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E33N17_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E33N18_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E34N17_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E34N18_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2018_010m_E30N16_03035_v020.tif', '../Data/Crop_Classification/forest_density_TCD/TCD_2

# Stitching all tiles

In [98]:
variables_des = [
    'NDVI_q3_median',
    'HP_prob',
    'SAVI_q3_median',
    'SAVI_amp5095',
    'NDVI_amp5095',
    'NDVI_p10',
    'SAR20_HV',
    'CO_prob',
    'AyF_prob',
    'NDVI_p20',
    'EOSD_20m',
    'elevation',
    'SAVI_p10',
    'NDMI_p95',
    'SAVI_p20',
    'forest_density_TCD',
    'NDMI_amp5095'
    ]

In [99]:
for variable in variables_des:
    src_files_to_mosaic = []
    var_list = glob(f'../Data/Crop_Classification/S2/{year}/*/{variable}.tif')
    print(variable, len(var_list))
    
    out_file  = f'../Data/Crop_Classification/{variable}.tif'
    
    if not path.isfile(out_file):
        if var_list:
            for var in var_list:
                src = rasterio.open(var)
                src_files_to_mosaic.append(src) 
        
            mosaic, out_trans = merge(src_files_to_mosaic)
    
            profile = src.meta.copy()
            profile.update({"driver": "GTiff",
                        "height": mosaic.shape[1],
                        "width": mosaic.shape[2],
                        "transform": out_trans,
                        "compress": "lzw"})
        
            with rasterio.open(out_file, "w", **profile) as dest:
                print(out_file)
                dest.write(mosaic)


NDVI_q3_median 6
HP_prob 6
SAVI_q3_median 6
SAVI_amp5095 6
NDVI_amp5095 6
NDVI_p10 6
SAR20_HV 0
CO_prob 6
AyF_prob 6
NDVI_p20 6
EOSD_20m 0
elevation 0
SAVI_p10 6
NDMI_p95 6
SAVI_p20 6
forest_density_TCD 0
NDMI_amp5095 6


# Create a stack of all variables

In [102]:
list_variables = glob(f'../Data/Crop_Classification/*.tif')
print(len(list_variables))

17


In [111]:

# Read metadata of first file
with rasterio.open(list_variables[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(list_variables),compress ='lzw', dtype= rasterio.float32)

# Read each layer and write it to stack
out_file = '../Data/Crop_Classification/Variables/Variables.tif'
if not path.isfile(out_file):
    with rasterio.open(out_file, 'w', **meta) as dst:
        for id, layer in enumerate(list_variables, start=1):
            with rasterio.open(layer) as src:
                print(layer)
                src = src.read(1).astype(np.float32)
                dst.write_band(id, src)
                dst.set_band_description(id, os.path.basename(layer).split()[-1][:-4])
                

../Data/Crop_Classification/elevation_GLO_25830.tif
../Data/Crop_Classification/SAR20_HV.tif
../Data/Crop_Classification/EOSD_20m.tif
../Data/Crop_Classification/forest_density_TCD.tif
../Data/Crop_Classification/NDVI_q3_median.tif
../Data/Crop_Classification/HP_prob.tif
../Data/Crop_Classification/SAVI_q3_median.tif
../Data/Crop_Classification/SAVI_amp5095.tif
../Data/Crop_Classification/NDVI_amp5095.tif
../Data/Crop_Classification/NDVI_p10.tif
../Data/Crop_Classification/CO_prob.tif
../Data/Crop_Classification/AyF_prob.tif
../Data/Crop_Classification/NDVI_p20.tif
../Data/Crop_Classification/SAVI_p10.tif
../Data/Crop_Classification/NDMI_p95.tif
../Data/Crop_Classification/SAVI_p20.tif
../Data/Crop_Classification/NDMI_amp5095.tif


# Resample raster to 10m for statistics (getting values for smaller parcels)

In [18]:
# specify input and output filenames
inputFile = '../Data/Crop_Classification/Variables/Variables.tif'
outputFile = '../Data/Crop_Classification/Variables/Variables_10.tif'

import gdal


xres=10
yres=10
resample_alg = 'near'

options = gdal.WarpOptions(xRes=xres, yRes=yres, resampleAlg=resample_alg, creationOptions = ['TFW=YES', 'COMPRESS=LZW'])


ds = gdal.Warp(outputFile, inputFile, options=options)
ds = None

In [61]:
bands = tuple(['elevation_GLO',
 'SAR20_HV',
 'EOSD_20m',
 'forest_density_TCD',
 'NDVI_q3_median',
 'HP_prob',
 'SAVI_q3_median',
 'SAVI_amp5095',
 'NDVI_amp5095',
 'NDVI_p10',
 'CO_prob',
 'AyF_prob',
 'NDVI_p20',
 'SAVI_p10',
 'NDMI_p95',
 'SAVI_p20',
 'NDMI_amp5095'])
 

In [ ]:
# Read Shape file
with fiona.open('../Data/Crop_Classification/Crop_parcel_2021/Cultivos_regados_AH2021_v6_20.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    print('Shape file Projection: ', shapefile.crs)

    
with rasterio.open('../Data/Crop_Classification/Variables/Variables.tif') as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta.copy()

# Save clipped imagery and change dtype
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 'dtype': 'float32',
                 'compress': "lzw",
                 'nodata':0})

print(out_meta)

img_trans_clip = os.path.join('../Data/Crop_Classification/Variables/Variables_clip.tif')   
with rasterio.open(img_trans_clip, "w", **out_meta) as dest:
    dest.descriptions = bands
    dest.write(out_image)
       


- Zonal statistics -  app. 2-3h

In [62]:
gdf_parcels = gpd.read_file(parcels)

output_stat_var = '../Data/Crop_Classification/Variables/crop_stat_median.geojson'

if not path.isfile(output_stat_var):
      with rasterio.open('../Data/Crop_Classification/Variables/Variables_clip.tif') as src:
            
            # Calculate statistics for each band
            df_var = pd. DataFrame()
            affine = src.transform
            for band_nr, band_name in enumerate(bands, start=1):
                  array = src.read(band_nr)
                  print(band_name)
                  df_zonal_stats = pd.DataFrame(zonal_stats(gdf_parcels, array, affine=affine, stats="median"))
                  df = df_zonal_stats.rename(columns={'median': f'median_{band_name}'})
                  df_var = pd.concat([df_var, df], axis=1) 

      gdf_stat_var = pd.concat([gdf_parcels, df_var], axis=1) 
      gdf_stat_var.to_file(output_stat_var, driver="GeoJSON") 
      
      

elevation_GLO


c:\Users\ansu\Miniconda3\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


SAR20_HV


c:\Users\ansu\Miniconda3\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


EOSD_20m


...

c:\Users\ansu\Miniconda3\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


SAVI_p20


c:\Users\ansu\Miniconda3\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


NDMI_amp5095


c:\Users\ansu\Miniconda3\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


# Monthly NDVI and Zonal_stat
- monthly NDVI for each tile

In [ ]:
indices               = ['NDVI'] 


def stack_indices(tile):
    for index in indices:
        for month in months:
            file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*{index}.tif')
            print(tile, index, len(file_list), month)
            output_stack= f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_M{month}.tif'

            with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
        
            meta.update(count = len(file_list))
        
            if not path.isfile(output_stack):
                with rasterio.open(output_stack, 'w', **meta) as dst:
                    for id, layer in enumerate(file_list, start=1):
                        with rasterio.open(layer) as src1:
                            dst.write_band(id, src1.read(1))
                            meta = src1.meta
                            print(id, src1)
            print('Done', output_stack)

            output_mean = f'../Data/Crop_Classification/S2/{year}/{tile}/NDVI_M{month}_mean.tif'
            output_stack= glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_M{month}.tif')
            
            if not path.isfile(output_mean):
                print('Working', output_mean)
                ds  = xr.open_rasterio(output_stack[0])
                mean = ds.mean(dim='band')
                mean.rio.to_raster(output_mean)   
                print(output_mean)
                ds = gdal.Open(output_mean, 1)
                sr = osr.SpatialReference()
                sr.ImportFromEPSG(25830)
                ds.SetProjection(sr.ExportToWkt())
                ds = None
            
            print('Done', output_mean)


def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_indices, tiles)
        

        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")

- Merge all tiles together for each month

In [ ]:
for month in months:
    src_files_to_mosaic = []
    month_tiles_list = glob(f'../Data/Crop_Classification/S2/{year}/*/NDVI_M{month}_mean.tif')
    print(month_tiles_list)
    for file_month in month_tiles_list:
        src = rasterio.open(file_month)
        src_files_to_mosaic.append(src) 
        
    mosaic, out_trans = merge(src_files_to_mosaic)

    profile = src.meta.copy()
    profile.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans,
                    "compress": "lzw"})
    
    
    out_file  = f'../Data/Crop_Classification/S2/{year}/NDVI_M{month}_mean.tif'
    
    if not path.isfile(out_file):
        with rasterio.open(out_file, "w", **profile) as dest:
            print(out_file)
            dest.write(mosaic)



- Create a stack of monthly NDVI images

In [ ]:
list_NDVI = glob(f'../Data/Crop_Classification/S2/{year}/NDVI_M*_mean.tif')
print(list_NDVI, len(list_NDVI))
      
with rasterio.open(list_NDVI[0]) as src0:
    meta = src0.meta
    meta.update(count = len(list_NDVI))
        
    output = f'../Data/Crop_Classification/S2/{year}/stack_month_NDVI.tif'

    if not path.isfile(output):
        with rasterio.open(output, 'w', **meta) as dst:
            for id, layer in enumerate(list_NDVI, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
                        meta = src1.meta
                        print(id, src1)
                    
print(output)
        
        

- Zonal Stats NDVI

In [63]:
output_stat_NDVI = '../Data/Crop_Classification/Variables/crop_stat_NDVI_median.geojson'

if not path.isfile(output_stat_NDVI):
      with rasterio.open(f'../Data/Crop_Classification/S2/{year}/stack_month_NDVI.tif') as src:
            df_stat_NDVI = pd. DataFrame()
            affine = src.transform
            for band in range(1, src.count+1):
                  print(band)
                  array = src.read(band)
                  df_zonal_stats = pd.DataFrame(zonal_stats(gdf_parcels, array, affine=affine, stats="median"))
                  df = df_zonal_stats.rename(columns={'median': f'median_{band}'})
                  df_stat_NDVI = pd.concat([df_stat_NDVI, df], axis=1)

      gdf_stat_NDVI = pd.concat([gdf_parcels, df_stat_NDVI], axis=1) 
      gdf_stat_NDVI.to_file(output_stat_NDVI, driver="GeoJSON") 

# Merge two data frames & Reclass to the training dataset

In [64]:
gdf_stat_NDVI.iloc[:, 21:81]

,min_1,max_1,mean_1,count_1,median_1,min_2,max_2,mean_2,count_2,median_2,...,min_11,max_11,mean_11,count_11,median_11,min_12,max_12,mean_12,count_12,median_12
0,0.030795,0.030795,0.030795,1,0.030795,0.276709,0.276709,0.276709,1,0.276709,...,0.272253,0.272253,0.272253,1,0.272253,0.241663,0.241663,0.241663,1,0.241663
1,-0.000818,-0.000818,-0.000818,1,-0.000818,0.233891,0.233891,0.233891,1,0.233891,...,0.236289,0.236289,0.236289,1,0.236289,0.191431,0.191431,0.191431,1,0.191431
2,0.068394,0.068394,0.068394,1,0.068394,0.388719,0.388719,0.388719,1,0.388719,...,0.415532,0.415532,0.415532,1,0.415532,0.390508,0.390508,0.390508,1,0.390508
3,0.005493,0.020658,0.013076,2,0.013076,0.274430,0.366182,0.320306,2,0.320306,...,0.359775,0.441085,0.400430,2,0.400430,0.372625,0.401080,0.386852,2,0.386852
4,0.102683,0.102683,0.102683,1,0.102683,0.293749,0.293749,0.293749,1,0.293749,...,0.414202,0.414202,0.414202,1,0.414202,0.296779,0.296779,0.296779,1,0.296779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277690,0.074856,0.553023,0.234221,485,0.232465,0.093028,0.438367,0.238600,485,0.232324,...,0.026521,0.540800,0.189617,485,0.158832,0.028204,0.437217,0.145971,485,0.124439
277691,0.088062,0.272912,0.167172,184,0.166608,0.089788,0.259710,0.156765,184,0.150723,...,0.158556,0.465226,0.295388,184,0.301257,0.072706,0.358792,0.138840,184,0.133257
277692,0.084880,0.246004,0.170934,141,0.178115,0.074776,0.227954,0.145286,141,0.143732,...,0.138066,0.736644,0.480189,141,0.500174,0.068730,0.357981,0.223747,141,0.225438
277693,0.106953,0.248247,0.173538,82,0.171182,0.103406,0.204252,0.144590,82,0.142682,...,0.236769,0.778121,0.484831,82,0.478351,0.125544,0.397986,0.223838,82,0.217429


In [8]:
gdf_stat_var = gpd.read_file(output_stat_var)
gdf_stat_NDVI = gpd.read_file(output_stat_NDVI)


# gdf_stat_var = gpd.read_file('../Data/Crop_Classification/Variables/crop_stat_median.geojson')
# gdf_stat_NDVI = gpd.read_file('../Data/Crop_Classification/Variables/crop_stat_NDVI_median.geojson')

gdf_stat_merge = pd.concat([gdf_stat_var, gdf_stat_NDVI.iloc[:, 21:81]], axis=1) 

In [ ]:
gdf_stat_merge['Class_num'] = gdf_stat_merge['C_AH2021']
gdf_stat_merge['Class_name'] = gdf_stat_merge['C_AH2021']
gdf_stat_merge['Class'] = gdf_stat_merge['C_AH2021']


gdf_stat_merge['Class_name'] = gdf_stat_merge['Class'].replace({'HU':'Family gardens' ,
                                                                'HV':'Herbaceous irrigated in summer',
                                                                'HV2':'Herbaceous with double harvest (including one in summer)',
                                                                'HV3':'Herbaceous with triple harvest (including one in summer)',
                                                                'HO':'Herbaceous irrigated in autumn',
                                                                'H2':'Herbaceous with double harvest (outside summer)', 
                                                                'H3':'Herbaceous with triple harvest (outside summer)',
                                                                'HI':'Young Herbaceous irrigated in winter',
                                                                'HP':'Herbaceous irrigated in spring', 
                                                                'ALM':'Almonds',
                                                                'INV':'Greenhouse',
                                                                'CIT':'Citrus',
                                                                'OV':'Olivos',
                                                                'FJ':'Young Fruit',
                                                                'LBC':'Low Density Tree Cover', 
                                                                'CITj':'Young Citrus',
                                                                'VP':'Vineyard (table grape)',
                                                                'FR':'Fruit', 
                                                                'V':'Vineyard'})

gdf_stat_merge['Class_num'] = gdf_stat_merge['Class_num'].replace({'HU':'13' ,
                                                                   'HV':'1',
                                                                   'HV2':'1',
                                                                   'HV3':'1',
                                                                   'HO':'1',
                                                                   'H2':'1', 
                                                                   'H3':'1',
                                                                   'HI':'1',
                                                                   'HP':'1', 
                                                                   'ALM':'6',
                                                                   'INV':'9',
                                                                   'CIT':'3',
                                                                   'OV':'4',
                                                                   'FRj':'10',
                                                                   'LBC':'12', 
                                                                   'CITj':'11',
                                                                   'VP':'8',
                                                                   'FR':'2', 
                                                                   'V':'7'})
                                             
 
gdf_stat_merge.Class = np.where(((gdf_stat_merge.C21_M05.eq('LAC')) & (gdf_stat_merge.Class_name.eq('Olivos'))),'OV_LAC', gdf_stat_merge.Class)
gdf_stat_merge.Class = np.where(((gdf_stat_merge.C21_M05.eq('LBC')) & (gdf_stat_merge.Class_name.eq('Olivos'))),'OV_LBC', gdf_stat_merge.Class)
gdf_stat_merge.Class_num = np.where((gdf_stat_merge.Class.eq('OV_LBC')),'5', gdf_stat_merge.Class_num)
gdf_stat_merge.Class_name = np.where((gdf_stat_merge.Class.eq('OV_LBC')),'Olivos LBC', gdf_stat_merge.Class_name)
gdf_stat_merge.Class_name = np.where((gdf_stat_merge.Class.eq('OV_LAC')),'Olivos LAC', gdf_stat_merge.Class_name)       


In [ ]:
gdf_stat_merge.groupby('Class_num').count()

In [ ]:

output_stat_all = '../Data/Crop_Classification/Variables/crop_stat_all.geojson'
gdf_stat_merge.to_file(output_stat_all, driver="GeoJSON") 

# Optional:  Calculation of Salinity index

- Normalized Difference Salinity Index (NDSI), *Khan 2005*
      -  ESI = expression((RED-NIR)/(RED+NIR)) 
   

In [2]:

def calc_salinity(tile):
    for month in months:
        files = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*REFL.tif')
        for file in files:   
            
            output_esi = ('{}_ESI.tif'.format(file.split('\\')[-1][:-9]))
        
            if not path.isfile(output_esi):
                with rasterio.open(file) as src: 

                    band_blue = src.read(1)/10000
                    band_red  = src.read(3)/10000
                    band_nir  = src.read(7)/10000

                    esi =  (band_red.astype(float)-band_nir.astype(float)) / (band_red.astype(float)+band_nir.astype(float))
                    kwargs = src.meta
                    kwargs.update(dtype=rasterio.float32,count = 1, compres ='lzw')

                    # Create the files
                    with rasterio.open(output_esi, 'w', **kwargs) as dst:
                        dst.write_band(1, esi.astype(rasterio.float32)) 
                    print(output_esi)

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(calc_salinity, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
        
        

NameError: name 'tiles' is not defined

NameError: name 'file' is not defined

-  Create a raster stack of ESI

In [6]:

def stack_esi(tile):
    for month in months:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*_ESI.tif')
      
        with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
        
        meta.update(count = len(file_list))
        
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/stack_month_{month}_ESI.tif'

        if not path.isfile(output):
            with rasterio.open(output, 'w', **meta) as dst:
                for id, layer in enumerate(file_list, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))

def run(tiles):
    with ThreadPool(processes=int(2)) as pool:
        return pool.map(stack_esi, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
        

Done


- Calculation of median per month

In [8]:

def sal_median (tile):
    for month in months:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/stack_month_{month}_ESI.tif')
        output = f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/month_{month}_ESI_median.tif'
        
        if not path.isfile(output):
            ds  = xr.open_rasterio(file_list[0])
            mean = ds.mean(dim='band')
            mean = mean.rio.write_crs(25830, inplace=True)   
            mean.rio.to_raster(output,compress='lzw')   
    

def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(sal_median, tiles)
         
if __name__ == "__main__":
    results = run(tiles)
    print("Done")
    

Done


- Check all stack raster 

In [9]:

for tile in tiles:
    list = []
    for month in months:
        file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/month_{month}_ESI_median.tif')
        list.extend(file_list)      
    print(tile, len(list))
    

30SWG 12
30SWH 12
30SXG 12
30SXH 12
30SXJ 12
30SYH 12


- Merge all tiles together for each month

In [12]:
for month in months:
    src_files_to_mosaic = []
    month_tiles_list = glob(f'../Data/Crop_Classification/S2/{year}/*/{month}/month_{month}_ESI_median.tif')
    print(month_tiles_list)
    for file_month in month_tiles_list:
        src = rasterio.open(file_month)
        src_files_to_mosaic.append(src) 
        
    mosaic, out_trans = merge(src_files_to_mosaic)

    profile = src.meta.copy()
    profile.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans,
                    "compress": "lzw"})
    

    out_file  = f'../Data/Crop_Classification/S2/{year}/salinity/ESI_{month}_{year}.tif'
    
    if not path.isfile(out_file):
        with rasterio.open(out_file, "w", **profile) as dest:
            print(out_file)
            dest.write(mosaic)

['../Data/Crop_Classification/S2/2021/30SWG/01/month_01_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SWH/01/month_01_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXG/01/month_01_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXH/01/month_01_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXJ/01/month_01_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SYH/01/month_01_ESI_median.tif']
['../Data/Crop_Classification/S2/2021/30SWG/02/month_02_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SWH/02/month_02_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXG/02/month_02_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXH/02/month_02_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SXJ/02/month_02_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SYH/02/month_02_ESI_median.tif']
['../Data/Crop_Classification/S2/2021/30SWG/03/month_03_ESI_median.tif', '../Data/Crop_Classification/S2/2021/30SWH/03/month_03_ESI_me

- Clip a Salinity Month raster by a tighter area

In [8]:

option = gdal.WarpOptions(cutlineDSName='../Data/AOI/Rio_Segura_AOI_Tighter25830.geojson', cropToCutline=True, dstNodata=0)

for month in months:
    files = glob(f'../Data/Crop_Classification/S2/2021/salinity/ESI_{month}_{year}.tif')
    for file in files:
        print(file)
        out_file =  f'../Data/Crop_Classification/S2/2021/salinity/ESI_{month}_{year}_clip.tif'
        sr = osr.SpatialReference()
        sr.ImportFromEPSG(25830)
        ds = gdal.Open(file, 1)
        ds.SetProjection(sr.ExportToWkt())
        OutTile = gdal.Warp(f'../Data/Crop_Classification/S2/2021/salinity/ESI_{month}_{year}_clip.tif', ds, options=option)
        OutTile = None
 

../Data/Crop_Classification/S2/2021/salinity/ESI_01_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_02_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_03_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_04_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_05_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_06_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_07_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_08_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_09_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_10_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_11_2021.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_12_2021.tif


- Clip by a Parcel SHP (10m buffer) 

In [11]:

buffer = 20

output_parcel_buff = parcels[:-4] + f'_{buffer}.shp'
parcels_gpd = gpd.read_file(parcels)
print(output_parcel_buff)

parcels_buffer= parcels_gpd.buffer(buffer)
parcels_buffer.to_file(output_parcel_buff)



In [17]:
with fiona.open(output_parcel_buff, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]


for month in months:
    files = glob(f'../Data/Crop_Classification/S2/2021/salinity/ESI_{month}_{year}.tif')
    for file in files:
        with rasterio.open(file) as src:
            out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            out_meta = src.meta
            
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

    out_file = f'../Data/Crop_Classification/S2/2021/salinity/ESI_{month}_{year}_parcel_10.tif'   

        
    with rasterio.open(out_file, "w", **out_meta) as dest:
            dest.nodata = 0
            dest.write(out_image)
            
    print(out_file)
    ds = gdal.Open(out_file, 1)
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(25830)
    ds.SetProjection(sr.ExportToWkt())
    ds = None
    sr = None
                        

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:56800 remote=tcp://127.0.0.1:32799>


../Data/Crop_Classification/S2/2021/salinity/ESI_01_2021_parcel_10.tif


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:59790 remote=tcp://127.0.0.1:32799>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:60012 remote=tcp://127.0.0.1:32799>


../Data/Crop_Classification/S2/2021/salinity/ESI_02_2021_parcel_10.tif


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:49930 remote=tcp://127.0.0.1:40895>


...

../Data/Crop_Classification/S2/2021/salinity/ESI_08_2021_parcel_10.tif


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:56848 remote=tcp://127.0.0.1:42469>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:56882 remote=tcp://127.0.0.1:42469>


../Data/Crop_Classification/S2/2021/salinity/ESI_09_2021_parcel_10.tif


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34098 remote=tcp://127.0.0.1:32799>


../Data/Crop_Classification/S2/2021/salinity/ESI_10_2021_parcel_10.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_11_2021_parcel_10.tif
../Data/Crop_Classification/S2/2021/salinity/ESI_12_2021_parcel_10.tif


# Monthly NDVI

- monthly NDVI for each tile

In [ ]:
indices               = ['NDVI'] 

def stack_indices(tile):
    for index in indices:
        for month in months:
            file_list = glob(f'../Data/Crop_Classification/S2/{year}/{tile}/{month}/*{index}.tif')
            print(tile, index, len(file_list), month)
            output_stack= f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_M{month}.tif'

            with rasterio.open(file_list[0]) as src0:
                meta = src0.meta
        
            meta.update(count = len(file_list))
        
            if not path.isfile(output_stack):
                with rasterio.open(output_stack, 'w', **meta) as dst:
                    for id, layer in enumerate(file_list, start=1):
                        with rasterio.open(layer) as src1:
                            dst.write_band(id, src1.read(1))
                            meta = src1.meta
                            print(id, src1)
            print('Done', output_stack)

            output_mean = f'../Data/Crop_Classification/S2/{year}/{tile}/NDVI_M{month}_mean.tif'
            output_stack= glob(f'../Data/Crop_Classification/S2/{year}/{tile}/stack_{index}_M{month}.tif')
            
            if not path.isfile(output_mean):
                print('Working', output_mean)
                ds  = xr.open_rasterio(output_stack[0])
                mean = ds.mean(dim='band')
                mean.rio.to_raster(output_mean)   
                print(output_mean)
                ds = gdal.Open(output_mean, 1)
                sr = osr.SpatialReference()
                sr.ImportFromEPSG(25830)
                ds.SetProjection(sr.ExportToWkt())
                ds = None
            
            print('Done', output_mean)


def run(tiles):
    with ThreadPool(processes=int(1)) as pool:
        return pool.map(stack_indices, tiles)
        
if __name__ == "__main__":
    results = run(tiles)
    print("Done")

- Merge all tiles together for each month

In [ ]:
for month in months:
    src_files_to_mosaic = []
    month_tiles_list = glob(f'../Data/Crop_Classification/S2/{year}/*/NDVI_M{month}_mean.tif')
    print(month_tiles_list)
    for file_month in month_tiles_list:
        src = rasterio.open(file_month)
        src_files_to_mosaic.append(src) 
        
    mosaic, out_trans = merge(src_files_to_mosaic)

    profile = src.meta.copy()
    profile.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans,
                    "compress": "lzw"})
    

    out_file  = f'../Data/Crop_Classification/S2/{year}/NDVI_M{month}_mean.tif'
    
    if not path.isfile(out_file):
        with rasterio.open(out_file, "w", **profile) as dest:
            print(out_file)
            dest.write(mosaic)



- Create a stack of monthly NDVI images

In [ ]:
list_NDVI = glob(f'../Data/Crop_Classification/S2/{year}/NDVI_M*_mean.tif')
print(list_NDVI, len(list_NDVI))
      
with rasterio.open(list_NDVI[0]) as src0:
    meta = src0.meta
    meta.update(count = len(list_NDVI))
        
    output = f'../Data/Crop_Classification/S2/{year}/stack_month_NDVI.tif'

    if not path.isfile(output):
        with rasterio.open(output, 'w', **meta) as dst:
            for id, layer in enumerate(list_NDVI, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
                        meta = src1.meta
                        print(id, src1)
                    
print(output)
       

- calculate statistics of NDVI

In [166]:

with rasterio.open(f'../Data/Crop_Classification/S2/{year}/stack_month_NDVI.tif') as src:
      df_stat_NDVI = pd. DataFrame()
      affine = src.transform
      output = '../Data/Crop_Classification/Variables/crop_stat_NDVI.geojson'
      if not path.isfile(output):
            for band in range(1, src.count):
                  print(band)
                  array = src.read(band)
                  df_zonal_stats = pd.DataFrame(zonal_stats(gdf_parcels, array, affine=affine, stats="mean median min max count"))
                  df = df_zonal_stats.rename(columns={'mean': f'mean_{band}', 'median': f'median_{band}', 'min': f'min_{band}','max': f'max_{band}','count': f'count_{band}'})
                  df_stat_NDVI = pd.concat([df_stat_NDVI, df], axis=1)

            gdf_stat_NDVI_parcels = pd.concat([gdf_parcels, df_stat_NDVI], axis=1)
            gdf_stat_NDVI_parcels.to_file(output, driver="GeoJSON") 


# Reclass the training dataset

In [167]:
gdf_stat_NDVI_test = gdf_stat_NDVI